In [1]:
from __future__ import print_function

import os, sys
import re
import cPickle, gzip
import progressbar
from rdkit.Chem import AllChem

data_directory = 'data'
data_filename = '2001-2013_USPTOapplications_reactionSmiles_feb2014filters.rsmi'
removed_mapping_filename = '2001-2013_rsmi_filtered_removed_mapping.pkl.gz'
length_data_filename = 'length_data.pkl.gz'

data_filepath = os.path.join(data_directory, data_filename)
removed_mapping_filepath = os.path.join(data_directory, removed_mapping_filename)
length_data_filepath = length_data_filename

length_list = [] # list of tuple (reactants length, products length)
num_lines = sum(1 for line in open(data_filepath, 'r')) # total lines
bar = progressbar.ProgressBar(redirect_stdout=True, max_value=num_lines)

with open(data_filepath, 'r') as data_file:
    with gzip.open(removed_mapping_filepath, 'wb+') as output_file:
        for i, line in enumerate(data_file):
            rsmi = re.split(re.compile(r'[ \t]'), line)
            rxn = AllChem.ReactionFromSmarts(rsmi[0], useSmiles=True)
            AllChem.RemoveMappingNumbersFromReactions(rxn)
            output_smiles = AllChem.ReactionToSmiles(rxn)
            length_list.append((output_smiles.rfind('>'), \
                                len(output_smiles)-output_smiles.rfind('>')-1))
            cPickle.dump((output_smiles, rsmi[2]+rsmi[1]), output_file, 2)
            bar.update(i)

# save length tuple list for later use
with open(length_data_filename, 'wb+') as f:
    cPickle.dump(length_list, f, 2)

 99% (1196008 of 1196165) |################ | Elapsed Time: 0:08:28 ETA: 0:00:00